# 📢📢📢 Projet NLP : La parole aux citoyens (Make.org - Le tourisme vert en Ille-et-Vilaine)📢📢📢
# Techniques de clustering
## Bibliothèques nécessaires

In [1]:
from collections import Counter

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_score

## Lecture des données

In [2]:
# Chargement des jeux de données
df = pd.read_parquet(
    "../data/processed/req_tourisme_responsable_embeddings.parquet", engine="pyarrow"
)

df.head(3)

,content,agree_count,disagree_count,neutral_count,agree_score,disagree_score,neutral_score,doNotCare,doNotUnderstand,doable,...,nb_exclamation,tokens,case,tokens_normalized,stop_words,tokens_final,tokens_lemmatized,embedding_prop,embedding_prop_x,embedding_prop_y
0,Il faut une surveillance des lieux touristique...,27,7,7,0.65,0.18,0.17,1,0,8,...,0,"['Il', 'faut', 'une', 'surveillance', 'des', '...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['une', 'surveillance', 'des', 'lieux', 'touri...","['il', 'une', 'des', 'avec', 'la', 'pour', 'ou']","[surveillance, lieux, amende, clef, degradatio...","['surveillance', 'lieux', 'touristique', 'amen...","[0.026475444436073303, 0.013547265902161598, 0...",60.746189,15.809195
1,Il faut donner des sacs poubelles et des cendr...,56,18,15,0.62,0.19,0.19,0,0,21,...,0,"['Il', 'faut', 'donner', 'des', 'sacs', 'poube...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['donner', 'des', 'sacs', 'poubelles', 'et', '...","['il', 'des', 'et', 'des', 'de', 'aux', 'des',...","[donner, sac, poubelle, cendrier, poch, vacanc...","['donner', 'sac', 'poubelle', 'cendrier', 'poc...","[0.034178830683231354, 0.05750420689582825, 0....",54.815662,12.159612
2,Il faut récompenser et mettre en avant les act...,35,7,14,0.62,0.13,0.25,0,5,12,...,0,"['Il', 'faut', 'récompenser', 'et', 'mettre', ...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['recompenser', 'et', 'mettre', 'en', 'avant',...","['il', 'et', 'en', 'les', 'des', 'du', 'et', '...","[recompenser, mettre, avant, acte, projet, eco...","['recompenser', 'mettre', 'avant', 'acte', 'pr...","[-0.011080465279519558, 0.053166456520557404, ...",-22.491800,47.501347


## Classification non-supervisé : première méthode, les K-Means
### Définition du nombre de clusters

In [3]:
# Définir le nombre de clusters
nombre_de_clusters = 10

# Initialiser et entraîner le modèle K-means
kmeans = KMeans(n_clusters=nombre_de_clusters, random_state=42)

### Création d'une matrice d'embeddings

In [4]:
# Convertir la colonne 'embedding_prop' en une seule matrice
embedding_matrix = np.stack(df["embedding_prop"].values)
embedding_matrix.shape

(1493, 20)

### Algorithme des K plus proche voisins

In [5]:
kmeans.fit(embedding_matrix)

Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "c:\Dev\envs\tourisme-vert-env\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Dev\envs\tourisme-vert-env\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Dev\envs\tourisme-vert-env\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Dev\envs\tourisme-vert-env\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x82 in position 101: invalid start byte


KMeans(n_clusters=10, random_state=42)

### Labelisation des phrases (assignation des clusters)

In [6]:
# Obtenir les labels de cluster pour chaque phrase
labels = kmeans.labels_

# Évaluer la qualité des clusters avec la métrique silhouette
silhouette_avg = silhouette_score(embedding_matrix, labels)
print("Silhouette Score:", silhouette_avg)

# Assigner les labels de cluster à chaque ligne de votre dataframe
df["cluster_label"] = labels

Silhouette Score: 0.09877496608915598


In [7]:
df["cluster_label"].value_counts()

cluster_label
3    255
8    231
2    182
1    174
9    159
4    132
0    120
7    109
5     96
6     35
Name: count, dtype: int64

### Affichage de quelques phrases pour déterminer une thématique globale

In [8]:
# Définir la largeur d'affichage souhaitée (par exemple, 1000)
largeur_affichage = 2000

# Appliquer la nouvelle largeur d'affichage
pd.set_option("display.max_colwidth", largeur_affichage)

In [9]:
df["content"][df["cluster_label"] == 0].sample(20)

1013                      Il faut que les PLU incitent à la construction et à la mise en place de solutions environnementales réelles et accessibles
712        Il faut mette des quotas d'accès aux lieux touristiques pour lutter contre le tourisme de masse et les dégradations/pollution venant avec
473         Il faut développer le tourisme dans des écovillages respectueux de l'environnement et dans des espaces naturels où ils seront attractifs
289               Il faut cesser de faire débuter et terminer les locations saisonnières hebdomadaires le samedi pour diminuer bouchons et pollution
1346          Il faut limiter le flux de touristes en été sur les plages et garder le littoral propre. Rendre impossible la venue sur certains sites
1286                       Il faut réviser ou créer des jauges de personnes pouvant accéder à un site naturel par an ou par jour pour les préserver.
71                  Il faut établir des quotas à la journée pour limiter le nombre de visiteurs pour les m

Le tourisme de masse, ses effets et conséquences

In [10]:
df["content"][df["cluster_label"] == 1].sample(20)

308         Il faut réduire le prix du transport ferroviaire pour que le choix de la voiture pour partir en vacances ne soit pas purement économique
794                                                  Il faut compenser nos émissions carbone quand nous empruntons des moyens de transport polluant.
143                                     Il faut interdire les petits vols en avion lorsqu'une autre alternative de transport (comme le train) existe
949                                                                                                    Il faut rendre accessible le centre Bretagne.
529                    Il faut valoriser l'utilisation du train en réduisant les prix, et réhabiliter les anciennes lignes ferroviaires abandonnées.
167                                                Il faut privilégier les transports en commun pour l'acces à des lieux touristiques très sensibles
748                 Il faut faciliter la mobilité / les déplacements au niveau national avec par exemple d

Le mode de transport responsable

Même chose pour les autres clusters. Avec ce qui suit, on peut aussi determiner une thématique avec les mots les plus fréquents.

### Représentation en 2D des clusters
Ajout des 15 mots les plus fréquents par cluster pour mieux indiquer la thématique

In [11]:
def get_top_words_by_cluster(
    df: pd.DataFrame, cluster_col: str, tokens_col: str, n: int = 15
) -> dict:
    # Dictionnaire pour stocker les mots les plus fréquents par cluster
    top_words_by_cluster = {}

    # Grouper les données par cluster
    grouped = df.groupby(cluster_col)

    for cluster_label, group in grouped:
        # Combiner tous les tokens du cluster
        all_tokens = [token for sublist in group[tokens_col] for token in sublist]

        # Compter les occurrences de chaque token
        token_counts = Counter(all_tokens)

        # Obtenir les n mots les plus fréquents
        top_words = token_counts.most_common(n)

        # Stocker les résultats
        top_words_by_cluster[cluster_label] = [word for word, _ in top_words]

    return top_words_by_cluster

In [12]:
# Obtenir les 15 mots les plus fréquents par cluster
top_words_by_cluster = get_top_words_by_cluster(
    df, "cluster_label", "tokens_final", n=15
)

In [13]:
# Ajouter une nouvelle colonne au DataFrame d'origine
df["top_15_words"] = df["cluster_label"].map(top_words_by_cluster)

In [14]:
df[["cluster_label", "top_15_words"]].head(3)

,cluster_label,top_15_words
0,9,"[dechet, lieux, tri, eau, poubelle, mettre, sensibiliser, eviter, produire, local, favoriser, accueil, structure, environnement, vacance]"
1,7,"[camping, car, parking, lieux, aire, developper, afin, public, prendre, promouvoir, toilette, arreter, ville, piste, proposer]"
2,4,"[responsable, developper, rendre, lieux, acteur, habitant, durable, limiter, port, camping, environnement, modele, inciter, creer, nature]"


In [15]:
# Créer un graphique dynamique avec Plotly
fig = px.scatter(
    df[
        [
            "content",
            "embedding_prop_x",
            "embedding_prop_y",
            "cluster_label",
            "top_15_words",
        ]
    ],
    x="embedding_prop_x",
    y="embedding_prop_y",
    hover_name="content",
    hover_data={"top_15_words": True},
    color="cluster_label",
    color_continuous_scale="Viridis",  # Choisir une palette de couleurs
    range_color=[
        0,
        df["cluster_label"].max(),
    ],  # Définir la plage de couleurs en fonction des étiquettes de cluster
    labels={"cluster_label": "Cluster Label"},  # Renommer l'étiquette de la légende
    color_continuous_midpoint=int(
        df["cluster_label"].max() / 2
    ),  # Point médian de la palette de couleurs
    opacity=0.7,
)
fig.update_traces(textposition="top center", showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    title=dict(
        text="2D Sentence Embeddings and Clusters (K-Means)",
        font=dict(size=25, color="black", family="Arial"),
        xanchor="center",
        yanchor="top",
        x=0.5,
        y=0.95,
    ),
    template="simple_white",
    hoverlabel_font_size=16,
    coloraxis_showscale=False,
)

fig.show()

In [16]:
# Sauvegarder le graphique en tant que fichier HTML
pyo.plot(
    fig,
    filename="../outputs/2D Sentence Embeddings and Clusters (K-Means).html",
    auto_open=False,
)

'../outputs/2D Sentence Embeddings and Clusters (K-Means).html'

## Classification non-supervisé : deuxième méthode, la CAH

In [17]:
# Créer un objet de classification ascendante hiérarchique
agg_clustering = AgglomerativeClustering(n_clusters=10)

# Adapter le modèle aux données
agg_clustering.fit(embedding_matrix)

# Obtenir les étiquettes des clusters
cluster_labels = agg_clustering.labels_

# Évaluer la qualité des clusters avec la métrique silhouette
silhouette_avg = silhouette_score(embedding_matrix, cluster_labels)
print("Silhouette Score:", silhouette_avg)

# Ajouter les étiquettes des clusters à votre dataframe
df["cluster_label_cah"] = cluster_labels

Silhouette Score: 0.06151732301899933


Le silhouette score est un peu moins bon pour la CAH. Mais, globalement, avec des vecteurs de mots, les clusters auront toujours beaucoup d'inertie intra-classe...

In [18]:
df["cluster_label_cah"].value_counts()

cluster_label_cah
1    294
3    256
8    210
5    210
4    168
0    136
2    130
9     44
6     44
7      1
Name: count, dtype: int64

In [19]:
df["content"][df["cluster_label_cah"] == 0].sample(20)

310                                Il faut privilégier et développer les modes de transport doux: marche à pied, vélo, transports collectifs locaux.
718                                                        Il faut développer les déplacements en train entre régions et supprimer les vols internes
1219     Il faut proposer des vélos (électriques ou non) à location à bas coût accessibles aux usagers des trains seulement pour promouvoir le train
100      Il faut favoriser le train pour les transports en faisant des réductions (au resto, au musée etc...) sous présentation des billets de train
921                         Il faut valoriser le train pour les destinations moins connues, rajouter des TER, le covoiturage en bus pour les plages.
536                         Il faut faire des pass voyages illimités transports bus/train à la semaine ou pour plusieurs semaines pour les touristes
707     Il faut sensibiliser et informer les voyageurs sur l’accès aux modes de transport locaux plus dura

Les mobilités douces, discussion sur le type de transport. Ressemble beaucoup au cluster 1 du K-Means

In [20]:
# Obtenir les 15 mots les plus fréquents par cluster
top_words_by_cluster = get_top_words_by_cluster(
    df, "cluster_label_cah", "tokens_final", n=15
)

In [21]:
# Ajouter une nouvelle colonne au DataFrame d'origine
df["top_15_words_cah"] = df["cluster_label_cah"].map(top_words_by_cluster)

In [22]:
df[["cluster_label_cah", "top_15_words_cah"]].head(3)

,cluster_label_cah,top_15_words_cah
0,4,"[camping, dechet, eau, lieux, car, poubelle, tri, produire, local, favoriser, aire, sensibiliser, accueil, environnement, vacance]"
1,4,"[camping, dechet, eau, lieux, car, poubelle, tri, produire, local, favoriser, aire, sensibiliser, accueil, environnement, vacance]"
2,1,"[mettre, responsable, pouvoir, etre, creer, grand, favoriser, acteur, place, nombre, environnement, rendre, durable, impact, developper]"


In [23]:
# Créer un graphique dynamique avec Plotly
fig = px.scatter(
    df[
        [
            "content",
            "embedding_prop_x",
            "embedding_prop_y",
            "cluster_label_cah",
            "top_15_words_cah",
        ]
    ],
    x="embedding_prop_x",
    y="embedding_prop_y",
    hover_name="content",
    hover_data={"top_15_words_cah": True},
    color="cluster_label_cah",
    color_continuous_scale="Viridis",  # Choisir une palette de couleurs
    range_color=[
        0,
        df["cluster_label_cah"].max(),
    ],  # Définir la plage de couleurs en fonction des étiquettes de cluster
    labels={"cluster_label_cah": "Cluster Label"},  # Renommer l'étiquette de la légende
    color_continuous_midpoint=int(
        df["cluster_label_cah"].max() / 2
    ),  # Point médian de la palette de couleurs
    opacity=0.7,
)
fig.update_traces(textposition="top center", showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    title=dict(
        text="2D Sentence Embeddings and Clusters (CAH)",
        font=dict(size=25, color="black", family="Arial"),
        xanchor="center",
        yanchor="top",
        x=0.5,
        y=0.95,
    ),
    template="simple_white",
    hoverlabel_font_size=16,
    coloraxis_showscale=False,
)

fig.show()

In [ ]:
# Sauvegarder le graphique en tant que fichier HTML
pyo.plot(
    fig,
    filename="../outputs/2D Sentence Embeddings and Clusters (CAH).html",
    auto_open=False,
)

'../outputs/2D Sentence Embeddings and Clusters (CAH).html'